# Assignment 2 - Feature extraction and classification

In this assignment, you are expected to

(1) extract global features from a publicly available dataset with one of the pre-trained neural networks available in pytorch, 

and 

(2) classify the dataset using the traditional k-Neural Neighbours classifier.

You are also asked to implement k-fold cross-validation to evaluate your model.

------------------------

In [ ]:
# Load needed packages
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

When working with Pytorch, dataloader() is a must to know function.

Read more about this function and the parameters it accepts in https://blog.paperspace.com/dataloaders-abstractions-pytorch/ ;

DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    collate_fn=None,
    pin_memory=False,
 )

In [ ]:
from torch.utils.data import DataLoader

The variable transform encapsulates the needed transformations of our data

Read more about transforms in https://blog.paperspace.com/dataloaders-abstractions-pytorch/

In [ ]:
transform = transforms.Compose([
    # resize
    transforms.Resize(32),
    # center-crop
    transforms.CenterCrop(32),
    # to-tensor
    transforms.ToTensor(),
    # normalize
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

--------------------------------

# INPUT DATASET

--------------------------------

Load your dataset

In [ ]:
# Example solution for the CIFAR dataset - Please, select the one you worked with in Assignment 1
dataset = 'CIFAR10'
classes = ('plane', 'car', 'bird', 
           'cat','deer', 'dog', 'frog', 
           'horse', 'ship', 'truck')


dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=False)

## Exercise: Dataset preparation

### Train - Test Split

Write a function **train_test_split(dataset, ratio)** which takes a dataset as an input and returns two datasets one for training and another for testing.


In [ ]:
def train_test_split(dataset, ratio):
    
    # your code here
    
    return training_data,testing_data

--------------------------------

# FEATURE EXTRACTION

Extract descriptros from the images in your train and test dataset. The dataset split should remain the same for all the experiments if you want to be fair when comparing performance.

--------------------------------

## Exercise: Feature 1 - RGB descriptros

Following the code you implement in Assignment 1, extract R, G, and B descriptros from the images and concatenate them to create a 1D feature vector of 24 values.

In [ ]:
# your code here

## Exercise: Feature 2 - Extract descriptors using pre-traind networks

Load pretrained a network to extract global features from the images. 
We will use the values of the last fully connected layer of the deep network as a descriptor, i.e. we will remove the last fully-connected layer. Therefore, after feed-fowarding the input through the network, we save the output as the descriptor of the image.

You can use different networks for this purpose.

Reading material to start with;

https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html#sphx-glr-beginner-transfer-learning-tutorial-py

https://medium.com/analytics-vidhya/cnn-transfer-learning-with-vgg-16-and-resnet-50-feature-extraction-for-image-retrieval-with-keras-53320c580853

In [ ]:
import torch.nn as nn
from torchvision import models

# name of the model you wish to use - it should be selected from this list
# [resnet, alexnet, vgg, squeezenet, densenet, inception]

------------------------

# PERFORMANCE EVALUATION

------------------------

## Exercise: Error function

Implement a function to evaluate the accuracy of your prediction. 
We will rely on the evaluation metric accuracy.

You are suggested to also use f-score, recall and precision. Have a look at https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html 

In [ ]:
def accuracy_metric(actual, predicted):
    
    # your code here
    
    return accuracy_value

--------------------------------

# TRAIN AND TEST YOUR MODEL

--------------------------------

## Exercise: k Nearest Neighbour model

Apply the classifier with different values of k (number of nearest neighbours) to the two sets of previously extracted descriptors (RGB and CNN features) and evaluate the performance of your models (accuracy).

You can have a look at the documentation to understand the parameters that define the learning of the model,
https://scikit-learn.org/stable/modules/neighbors.html


In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [ ]:
# Use your k-NN - play with the value of the parameters to see how the model performs
kvalue_list = [2,4,6,10,15] 

# YOUR CODE GOES HERE

## Exercise: Visualize results 

Steps to follow:

1) Apply PCA and select the 2 first principal components to represent each sample.

2) Plot the samples with dots. Use a colour per class. 

3) Plot the samples again but with empty filled circles. Use the color of the class predicted per sample (misclassifications will make the colours not coincide).

You can do this for (1) training and (2) test set. In (1) you can see how well the method fits the training data and (2) will give you an idea of the misclassifications.

In [ ]:
# your code here

## Exercise: kNN with k-Fold cross-validation

Assess the performance of your implemented kNN using k-Fold cross-validation. 

Run your implemented function evaluating for k (fold) = 2, 5 and 10. You can rely on the kNN that performed best in the previous exercises.
Report the average accuracy and the standard deviation.

In [ ]:
# Load packages
from sklearn.model_selection import KFold
import numpy as np
from sklearn.utils import shuffle

# your code here

In [ ]:
## SUGGESTION ON HOW TO PRESENT PERFORMANCE OF YOUR KFOLD CROSS VALIDATION ANALYSIS

print('Summary results:')
print(' ')
print(' ')
for i,k in enumerate(k_list):
    print(k,'-fold cross validation:')  
    print('Accuracies per fold: ', avg_acc_list[i]) 
    
    avg_acc = round(sum(avg_acc_list[i])/k,2)
    std_list= round(np.std(avg_acc_list[i]),2)
    print('Average accuracy: ', avg_acc,'+-', std_list) 
    print(' ')

### [Optional] Exercise: further explore by: 
- implement other classifiers such as SVM or Random Forest, 
- extract other descriptors from the images such as objects or other local features,
- implement the evaluation metrics: recall, precission and f-score.